In [9]:
%pip install -q pytidb "pytidb[models]" openai python-dotenv pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
import uuid
import dotenv
from datetime import datetime, timedelta
from pytidb import TiDBClient
from openai import OpenAI
from pytidb.embeddings import EmbeddingFunction
from pytidb.schema import TableModel, Field

dotenv.load_dotenv()


# 🧠 Set up the embedding function
text_embed = EmbeddingFunction("openai/text-embedding-3-small")

import os

db = TiDBClient.connect(
    host=os.getenv("TIDB_HOST"),
    username=os.getenv("TIDB_USERNAME"),
    password=os.getenv("TIDB_PASSWORD"),
    database=os.getenv("TIDB_DATABASE"),
    port=int(os.getenv("TIDB_PORT"))
)

# 🧾 Define the shared memory table schema
class BotMemory(TableModel, table=True):
    __tablename__ = "memory"
    __table_args__ = {"extend_existing": True}

    id: int = Field(primary_key=True)
    agent_id: str = Field()
    memory_type: str = Field()
    content: str = Field()
    embedding: list[float] = text_embed.VectorField(source_field="content")
    tags: str = Field()
    visibility: str = Field()
    expires_at: datetime = Field()
    created_at: datetime = Field(default_factory=datetime.now)

# 🔧 Create the table (if not already created)
memory_table = db.create_table(schema=BotMemory)

In [11]:

memory_table.truncate()  # Optional: clear old data for clean test

# 🚀 Seed shared memory with example cases
print(str(uuid.uuid4()))
memories = [
    BotMemory(
        id=1,
        agent_id="agent_a",  # BillingBot
        memory_type="case",
        content="Refund processed for failed transaction.",
        tags='{"topic": "refund"}',
        visibility="team",
        expires_at=datetime.now() + timedelta(days=30),
        created_at=datetime.now()
    ),
    BotMemory(
        id=2,
        agent_id="agent_a",
        memory_type="case",
        content="Steps to manually trigger refund if auto-refund fails.",
        tags='{"topic": "billing"}',
        visibility="team",
        expires_at=datetime.now() + timedelta(days=30),
        created_at=datetime.now()
    ),
    BotMemory(
        id=3,
        agent_id="agent_b",  # PolicyBot
        memory_type="policy",
        content="Policy allows refunds within 30 days unless caused by technical issues.",
        tags='{"topic": "policy"}',
        visibility="public",
        expires_at=datetime.now() + timedelta(days=30),
        created_at=datetime.now()
    )
]
memory_table.bulk_insert(memories)

48060c0f-2846-4252-8208-b7e7fa922a84


[BotMemory(id=1, content='Refund processed for failed transaction.', tags='{"topic": "refund"}', expires_at=datetime.datetime(2025, 5, 5, 23, 7, 11), memory_type='case', agent_id='agent_a', embedding=array([-0.03695381,  0.00903409, -0.04897521, ..., -0.01637316,
         0.00760955, -0.005055  ], dtype=float32), visibility='team', created_at=datetime.datetime(2025, 4, 5, 23, 7, 11)),
 BotMemory(id=2, content='Steps to manually trigger refund if auto-refund fails.', tags='{"topic": "billing"}', expires_at=datetime.datetime(2025, 5, 5, 23, 7, 11), memory_type='case', agent_id='agent_a', embedding=array([-0.04991492,  0.0643032 ,  0.0341817 , ..., -0.01978072,
         0.00199044, -0.01672289], dtype=float32), visibility='team', created_at=datetime.datetime(2025, 4, 5, 23, 7, 11)),
 BotMemory(id=3, content='Policy allows refunds within 30 days unless caused by technical issues.', tags='{"topic": "policy"}', expires_at=datetime.datetime(2025, 5, 5, 23, 7, 11), memory_type='policy', agent_

In [12]:
import pandas as pd

# 💬 Customer Query 1: goes to BillingBot
customer_query = "I was charged twice and didn’t receive a refund yet."

print("\n🙋‍♂️ [Customer] " + customer_query)

# BillingBot does vector search to build context
visibilities = ["team", "public"]
results = []

for v in visibilities:
    partial = (
        memory_table.search(customer_query)
        .filter({"visibility": v})  # ✅ Only a single string, not a list
        .limit(3)
        .to_pandas()
    )
    results.append(partial)

# Combine and remove duplicates
combined = pd.concat(results).drop_duplicates(subset=["content"]).head(3)

print("📥 [BillingBot] Retrieved shared memory for refund issue:")
for _, row in combined.iterrows():
    print("-", row["content"])

# (Optional) Simulate a simple RAG-generated response
print("\n🤖 [BillingBot Response]")
print("I see past cases where refunds were manually processed after sync issues.")
print("You can request a manual refund via the dashboard or I can trigger one now.")

# 💬 Customer Query 2: goes to PolicyBot
customer_query2 = "Can I still get a refund after 30 days if I was wrongly charged?"

print("\n🙋‍♂️ [Customer] " + customer_query2)

policy_results = (
    memory_table.search(customer_query2)
    .filter({"visibility": "public"}) 
    .limit(3)
    .to_pandas()
)

print("\n📥 [PolicyBot] Retrieved shared memory for refund policy:")
for _, row in policy_results.iterrows():
    print("🧠", row["content"])

print("\n🤖 [PolicyBot Response]")
print("Our policy allows refunds within 30 days, but we can make exceptions if it’s due to a technical error.")


🙋‍♂️ [Customer] I was charged twice and didn’t receive a refund yet.
📥 [BillingBot] Retrieved shared memory for refund issue:
- Refund processed for failed transaction.
- Steps to manually trigger refund if auto-refund fails.
- Policy allows refunds within 30 days unless caused by technical issues.

🤖 [BillingBot Response]
I see past cases where refunds were manually processed after sync issues.
You can request a manual refund via the dashboard or I can trigger one now.

🙋‍♂️ [Customer] Can I still get a refund after 30 days if I was wrongly charged?

📥 [PolicyBot] Retrieved shared memory for refund policy:
🧠 Policy allows refunds within 30 days unless caused by technical issues.

🤖 [PolicyBot Response]
Our policy allows refunds within 30 days, but we can make exceptions if it’s due to a technical error.
